In [12]:
import os

import numpy as np
import pandas as pd
import json
import cv2
from sklearn.model_selection import train_test_split
import cv2

# tf imports
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras import backend as K

In [13]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPU


loading images with labels

In [14]:
f = open("..\data\measurements.json")
data_json = json.loads(f.read())["Measurements"]

vs = data_json.values()

df = pd.json_normalize(vs)

df.head()


,Variety,RGB_Image,Depth_Information,FreshWeightShoot,DryWeightShoot,Height,Diameter,LeafArea
0,Lugano,RGB_27.png,Depth_27.png,5.5,0.42,9.0,11.0,153.9
1,Lugano,RGB_79.png,Depth_79.png,30.3,1.92,8.8,20.2,582.3
2,Lugano,RGB_140.png,Depth_140.png,60.9,2.83,11.6,19.0,960.6
3,Lugano,RGB_203.png,Depth_203.png,112.0,5.76,11.0,22.0,1614.9
4,Lugano,RGB_292.png,Depth_292.png,372.6,14.17,17.0,32.0,3839.6


In [57]:
import albumentations as alb

def augment(train_set, train_labels):
    # Define a list of image augmentation transformations to apply
    aug_transforms = [
        alb.HorizontalFlip(),
        alb.VerticalFlip(),
        alb.RandomRotate90(),
        alb.Transpose(),
        alb.RandomBrightnessContrast()
    ]

    # Use the Compose function to apply the transformations
    aug = alb.Compose(aug_transforms)

    # Set the number of times to augment the images
    num_augmentations = 1

    # Iterate through the number of augmentations
    for i in range(num_augmentations):
      # Iterate through your training set and apply the augmentations to each image
      augmented_train_set = []
      augmented_train_labels = []
      for image, label in zip(train_set, train_labels):
        augmented_image = aug(image=image)['image']
        augmented_train_set.append(augmented_image)
        augmented_train_labels.append(label)

      # Add the augmented images and labels to the training set
      train_set = np.concatenate((train_set, augmented_train_set))
      train_labels = np.concatenate((train_labels, augmented_train_labels))

    # Shuffle the training set and labels
    p = np.random.permutation(len(train_set))
    X = train_set[p]
    y = train_labels[p]
    return X, y


In [58]:
"""
    number: depth=0, RGB=1, depth/RGB=2
"""
def prepare_data(number=0, y_key='Height'):

    all_images = []
    y = []

    if number == 2:
        path_RGB = '../data/RGBImagesResize/'
        key_RGB = 'RGB_Image'
        flag_RGB = cv2.IMREAD_COLOR

        path_depth = '../data/DepthImagesResize/'
        key_depth = 'Depth_Information'
        flag_depth = cv2.IMREAD_GRAYSCALE

        for image_path_RGB in os.listdir(path_RGB):
            if df.loc[df[key_RGB] == image_path_RGB][y_key].size > 0:
                img_ID = image_path_RGB.strip(".png").strip("RGB_")
                y.append(df.loc[df[key_RGB] == image_path_RGB][y_key].item())
                img_RGB = cv2.imread(path_RGB + image_path_RGB, flag_RGB)
                img_RGB = cv2.resize(img_RGB, (250, 250))
                image_path_depth = path_depth + "Depth_"+ img_ID + ".png"
                img_depth = cv2.imread(image_path_depth, flag_depth)
                img_depth = cv2.resize(img_depth, (250,250))
                img_combined = np.dstack((img_RGB, img_depth))
                all_images.append(img_combined)
    else:
        if number == 1:
            path = '../data/RGBImagesResize/'
            key = 'RGB_Image'
            flag = cv2.IMREAD_COLOR
        else:
            path = '../data/DepthImagesResize/'
            key = 'Depth_Information'
            flag = cv2.IMREAD_GRAYSCALE

        for image_path in os.listdir(path):
            if df.loc[df[key] == image_path][y_key].size > 0:
                y.append(df.loc[df[key] == image_path][y_key].item())
                img = cv2.imread(path + image_path, flag)
                img = cv2.resize(img, (250, 250))
                all_images.append(img)


    X_train, X_test, y_train, y_test = train_test_split(np.array(all_images), y, test_size=0.25, shuffle=True)

    X_train, y_train = augment(X_train, y_train)

    X_train = tf.convert_to_tensor(X_train)
    X_test = tf.convert_to_tensor(X_test)
    y_train = tf.convert_to_tensor(y_train)
    y_test = tf.convert_to_tensor(y_test)

    return X_train, y_train, X_test, y_test

ResNet Implementation for regression: depth images height

https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba

!!LET OP NIET RUNNEN ZONDER GPU!!

In [59]:
from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [60]:
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

In [61]:
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides=(1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [62]:
def create_res_net(input_shape, regression):
    inputs = Input(shape=input_shape)
    num_filters = 64

    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)

    num_blocks_list = [2, 5, 5, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j == 0 and i != 0), filters=num_filters)
        num_filters *= 2

    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(19, activation='softmax')(t)

    if regression:
        outputs = Dense(1, activation='linear')(t)

    model = Model(inputs, outputs)

    if regression:
        model.compile(
            optimizer='adam',
            loss='mean_squared_error',
            metrics=['mean_absolute_percentage_error']
        )
    else:
        model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    return model

In [63]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import datetime
import os

def fit_model(model, epochs, batch_size, X_train, y_train, X_test, y_test):
    timestr = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    name = 'res_net'+timestr

    checkpoint_path = "checkpoints/"+name+"/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    os.system('mkdir {}'.format(checkpoint_dir))

    cp_callback = ModelCheckpoint(
        filepath=checkpoint_path,
        verbose=0,
        save_best_only=True,
        monitor='val_loss',
        mode='min'
    )
    tensorboard_callback = TensorBoard(
        log_dir=f'tensorboard_logs/{name}',
        histogram_freq=1
    )
    # early_stop_callback = EarlyStopping(
    #     monitor='val_loss',
    #     mode='min',
    #     patience=10
    # )

    model.fit(
        x=X_train,
        y=y_train,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test, y_test),
        batch_size=batch_size,
        callbacks=[cp_callback, tensorboard_callback]
    )

In [68]:
model = create_res_net((250,250,1),True)
X_train, y_train, X_test, y_test = prepare_data(0, 'Height')

In [69]:
fit_model(model, 100, 8, X_train, y_train, X_test, y_test)

Epoch 1/100
73/73 [==============================] - ETA: 0s - loss: 79.0476 - mean_absolute_percentage_error: 60.1966

INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0001.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0001.ckpt\assets


73/73 [==============================] - 68s 804ms/step - loss: 79.0476 - mean_absolute_percentage_error: 60.1966 - val_loss: 2057346.2500 - val_mean_absolute_percentage_error: 13506.9043
Epoch 2/100
73/73 [==============================] - 43s 583ms/step - loss: 53.5410 - mean_absolute_percentage_error: 52.4944 - val_loss: 495485216.0000 - val_mean_absolute_percentage_error: 214312.1406
Epoch 3/100
73/73 [==============================] - ETA: 0s - loss: 30.1873 - mean_absolute_percentage_error: 40.5782

INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0003.ckpt\assets


...

INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0060.ckpt\assets


73/73 [==============================] - 52s 716ms/step - loss: 10.7953 - mean_absolute_percentage_error: 22.7840 - val_loss: 5.9401 - val_mean_absolute_percentage_error: 16.9186
Epoch 61/100
73/73 [==============================] - 43s 580ms/step - loss: 9.9496 - mean_absolute_percentage_error: 22.3767 - val_loss: 10.0004 - val_mean_absolute_percentage_error: 17.8174
Epoch 62/100
73/73 [==============================] - 43s 581ms/step - loss: 10.6360 - mean_absolute_percentage_error: 22.4050 - val_loss: 6.0523 - val_mean_absolute_percentage_error: 18.1475
Epoch 63/100
73/73 [==============================] - 42s 579ms/step - loss: 10.2772 - mean_absolute_percentage_error: 22.9248 - val_loss: 11.4525 - val_mean_absolute_percentage_error: 18.7211
Epoch 64/100
73/73 [==============================] - ETA: 0s - loss: 10.6914 - mean_absolute_percentage_error: 23.5604

INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0064.ckpt\assets


INFO:tensorflow:Assets written to: checkpoints/res_net20221219-195506\cp-0064.ckpt\assets


73/73 [==============================] - 52s 718ms/step - loss: 10.6914 - mean_absolute_percentage_error: 23.5604 - val_loss: 4.8677 - val_mean_absolute_percentage_error: 15.0920
Epoch 65/100
73/73 [==============================] - 43s 587ms/step - loss: 11.1618 - mean_absolute_percentage_error: 24.2069 - val_loss: 6.1061 - val_mean_absolute_percentage_error: 18.8123
Epoch 66/100
73/73 [==============================] - 43s 590ms/step - loss: 10.9653 - mean_absolute_percentage_error: 23.4960 - val_loss: 20.4559 - val_mean_absolute_percentage_error: 34.1973
Epoch 67/100
73/73 [==============================] - 43s 588ms/step - loss: 10.5500 - mean_absolute_percentage_error: 22.8476 - val_loss: 13.2161 - val_mean_absolute_percentage_error: 31.8085
Epoch 68/100
73/73 [==============================] - 43s 583ms/step - loss: 9.6488 - mean_absolute_percentage_error: 22.2855 - val_loss: 17.4582 - val_mean_absolute_percentage_error: 37.6156
Epoch 69/100
73/73 [==============================]